In [ ]:
!sudo apt install tesseract-ocr
!pip install pytesseract

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 29 not upgraded.
Need to get 4,795 kB of archives.
After this operation, 15.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-eng all 4.00~git24-0e00fe6-1.2 [1,588 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-osd all 4.00~git24-0e00fe6-1.2 [2,989 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr amd64 4.00~git2288-10f4998a-2 [218 kB]
Fetched 4,795 kB in 1s (3,449 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl

In [ ]:
import numpy as np
import cv2
from imutils.object_detection import non_max_suppression
import pytesseract
import argparse
import time
import sys
from PIL import Image
from scipy.ndimage import interpolation as inter
from matplotlib import pyplot as plt
import tempfile

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving 10.png to 10.png


In [ ]:
image_path_in_colab="/content/8.png"

In [ ]:
args = {"image":"../input/text-detection/example-images/Example-images/ex24.jpg", "east":"../content/east_text_detection.pb", "min_confidence":0.5, "width":320, "height":320,"padding":0.0}

In [ ]:
IMAGE_SIZE = 1800


def set_image_dpi(file_path):

    im = Image.open(file_path)
    length_x, width_y = im.size
    factor = max(1, int(IMAGE_SIZE / length_x))
    size = factor * length_x, factor * width_y
    # size = (1800, 1800)
    im_resized = im.resize(size, Image.ANTIALIAS)
    temp_file = tempfile.NamedTemporaryFile(delete=False, suffix='.png')
    temp_filename = temp_file.name
    im_resized.save(temp_filename, dpi=(300, 300))
    return temp_filename
  

#img= cv2.imread(img)
#im = Image.open(img)

#show image
#im.show()

In [ ]:
#noise removal and smoothening
BINARY_THREHOLD = 180

def image_smoothening(img):
    ret1, th1 = cv2.threshold(img, BINARY_THREHOLD, 255, cv2.THRESH_BINARY)
    ret2, th2 = cv2.threshold(th1, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    blur = cv2.GaussianBlur(th2, (1, 1), 0)
    ret3, th3 = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return th3

def remove_noise_and_smooth(file_name):
    img = cv2.imread(file_name, 0)
    filtered = cv2.adaptiveThreshold(img.astype(np.uint8), 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 9, 41)
    kernel = np.ones((1, 1), np.uint8)
    opening = cv2.morphologyEx(filtered, cv2.MORPH_OPEN, kernel)
    closing = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel)
    img = image_smoothening(img)
    or_image = cv2.bitwise_or(img, closing)
    return or_image

In [ ]:
img_dpi = set_image_dpi(image_path_in_colab)

import cv2
import numpy as np

# read the image
img = cv2.imread(img_dpi)

# convert to gray
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

# apply morphology
kernel = cv2.getStructuringElement(cv2.MORPH_RECT , (3,3))
smooth = cv2.morphologyEx(gray, cv2.MORPH_DILATE, kernel)

# alternate blur in place of morphology
#smooth = cv2.GaussianBlur(gray, (15,15), 0)

# divide gray by morphology image
division = cv2.divide(gray, smooth, scale=255)

# threshold
result = cv2.threshold(division, 0, 255, cv2.THRESH_OTSU )[1] 

# save results
cv2.imwrite('img_thresh.png',result)



True

In [ ]:

import cv2
import numpy as np
from scipy.ndimage import interpolation as inter

def correct_skew(image, delta=1, limit=5):
    def determine_score(arr, angle):
        data = inter.rotate(arr, angle, reshape=False, order=0)
        histogram = np.sum(data, axis=1)
        score = np.sum((histogram[1:] - histogram[:-1]) ** 2)
        return histogram, score

    #img = cv2.imread(image, cv2.IMREAD_COLOR)


    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1] 

    scores = []
    angles = np.arange(-limit, limit + delta, delta)
    for angle in angles:
        histogram, score = determine_score(thresh, angle)
        scores.append(score)

    best_angle = angles[scores.index(max(scores))]

    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, best_angle, 1.0)
    rotated = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC, \
              borderMode=cv2.BORDER_REPLICATE)

    return best_angle, rotated



In [ ]:
img3 = cv2.imread(img_dpi)
angle, rotated = correct_skew(img3)

In [ ]:

extractedInformation = pytesseract.image_to_string(rotated)
print(extractedInformation)

Foreword

The need to get the best possible value from spending public money will
always remain a constant for those entrusted with spending decisions. The
need to reduce overall spending resulting from the financial crisis of 2008 has
sharpened this requirement. The continuing downward pressure on the
availability of public sector finance together with the ever growing upward
pressures of demand for public services will continue to further increase the
need to make better use of the resources available, the challenge has never
been greater.

In this context it is vital that capital spending decisions are taken on the basis
of highly competent professionally developed spending proposals. This
Treasury guidance which has been refined and tested over many years
provides a clear framework for thinking about spending proposals and a
structured process for appraising, developing and planning to deliver best
public value. All of which is captured through a well prepared business case
which s

In [ ]:
print(pytesseract.image_to_boxes(rotated))

F 79 1613 94 1645 0
o 94 1613 118 1645 0
r 118 1613 143 1645 0
e 143 1613 163 1645 0
w 163 1613 182 1645 0
o 182 1613 199 1645 0
r 208 1613 231 1646 0
d 231 1613 264 1646 0
T 79 1520 94 1551 0
h 94 1520 116 1551 0
e 116 1520 149 1551 0
n 168 1519 181 1551 0
e 181 1519 202 1551 0
e 202 1519 225 1551 0
d 225 1519 259 1551 0
t 279 1519 287 1547 0
o 287 1519 313 1547 0
g 335 1511 346 1544 0
e 346 1511 362 1544 0
t 379 1520 391 1547 0
t 410 1520 423 1551 0
h 423 1520 441 1551 0
e 441 1520 468 1551 0
b 486 1519 500 1551 0
e 500 1519 516 1551 0
s 516 1519 541 1551 0
t 541 1519 564 1551 0
p 584 1511 600 1545 0
o 600 1511 616 1545 0
s 616 1511 633 1545 0
s 633 1511 658 1545 0
i 658 1511 670 1545 0
b 675 1520 693 1551 0
l 693 1520 712 1551 0
e 712 1520 732 1551 0
v 753 1520 766 1543 0
a 774 1519 791 1551 0
l 806 1519 808 1545 0
u 808 1519 825 1545 0
e 825 1519 850 1545 0
f 870 1520 883 1551 0
r 895 1520 900 1545 0
o 900 1520 923 1545 0
m 923 1520 950 1545 0
s 971 1512 987 1551 0
p 987 1512 1010 

In [ ]:
file1 = open("myfile2.txt","w") 
file1.writelines(extractedInformation) 